In [ ]:
%%writefile utils_validation.py
import pandas as pd
from datasets import Dataset
from constants_validation import POSITIVE_ANSWER, NEGATIVE_ANSWER, COMPLETE_PHRASE, BASE_PROMPT, TRAIN_PERCENTAGE, VALIDATION_PERCENTAGE
import random, numpy as np
from sklearn.model_selection import train_test_split
random.seed(42)
np.random.seed(42)


def build_prompt(row):
    return f"""
{BASE_PROMPT}

Subreddit: r/{row["subreddit"]}
Rule: {row["rule"]}
Examples:
1) {row["positive_example"]}
{COMPLETE_PHRASE} Yes

2) {row["negative_example"]}
{COMPLETE_PHRASE} No

---
Comment: {row["body"]}
{COMPLETE_PHRASE}"""


def get_dataframe_to_train_validation(data_path, mode='train'):
    """
    Modified function to support train/validation split from training data
    mode: 'train' to get training portion, 'validation' to get validation portion
    """
    train_dataset = pd.read_csv(f"{data_path}/train.csv")
    test_dataset = pd.read_csv(f"{data_path}/test.csv").sample(frac=0.5, random_state=42).reset_index(drop=True)
    
    # Split training data into train and validation
    train_split, val_split = train_test_split(
        train_dataset, 
        test_size=VALIDATION_PERCENTAGE, 
        random_state=42, 
        stratify=train_dataset['rule_violation']
    )
    
    if mode == 'train':
        chosen_dataset = train_split
        print(f"Using {len(chosen_dataset)} samples for training ({TRAIN_PERCENTAGE*100:.1f}% of original training data)")
    elif mode == 'validation':
        chosen_dataset = val_split
        print(f"Using {len(chosen_dataset)} samples for validation ({VALIDATION_PERCENTAGE*100:.1f}% of original training data)")
    else:
        raise ValueError("mode must be 'train' or 'validation'")

    flatten = []

    # ---------- Process chosen dataset ----------
    chosen_df = chosen_dataset[["body", "rule", "subreddit", "rule_violation",
                              "positive_example_1","positive_example_2",
                              "negative_example_1","negative_example_2"]].copy()

    # Randomly select positive_example and negative_example
    chosen_df["positive_example"] = np.where(
        np.random.rand(len(chosen_df)) < 0.5,
        chosen_df["positive_example_1"],
        chosen_df["positive_example_2"]
    )
    chosen_df["negative_example"] = np.where(
        np.random.rand(len(chosen_df)) < 0.5,
        chosen_df["negative_example_1"],
        chosen_df["negative_example_2"]
    )

    # Drop original candidate columns
    chosen_df.drop(columns=["positive_example_1","positive_example_2",
                           "negative_example_1","negative_example_2"], inplace=True)

    flatten.append(chosen_df)

    # ---------- Process test dataset (only for training mode) ----------
    if mode == 'train':
        for violation_type in ["positive", "negative"]:
            for i in range(1, 3):
                sub_dataset = test_dataset[["rule","subreddit",
                                            "positive_example_1","positive_example_2",
                                            "negative_example_1","negative_example_2"]].copy()

                if violation_type == "positive":
                    # body uses current positive_example
                    body_col = f"positive_example_{i}"
                    other_positive_col = f"positive_example_{3-i}"  # other positive
                    sub_dataset["body"] = sub_dataset[body_col]
                    sub_dataset["positive_example"] = sub_dataset[other_positive_col]
                    # negative_example randomly selected
                    sub_dataset["negative_example"] = np.where(
                        np.random.rand(len(sub_dataset)) < 0.5,
                        sub_dataset["negative_example_1"],
                        sub_dataset["negative_example_2"]
                    )
                    sub_dataset["rule_violation"] = 1

                else:  # violation_type == "negative"
                    body_col = f"negative_example_{i}"
                    other_negative_col = f"negative_example_{3-i}"
                    sub_dataset["body"] = sub_dataset[body_col]
                    sub_dataset["negative_example"] = sub_dataset[other_negative_col]
                    sub_dataset["positive_example"] = np.where(
                        np.random.rand(len(sub_dataset)) < 0.5,
                        sub_dataset["positive_example_1"],
                        sub_dataset["positive_example_2"]
                    )
                    sub_dataset["rule_violation"] = 0

                # Drop original candidate columns
                sub_dataset.drop(columns=["positive_example_1","positive_example_2",
                                          "negative_example_1","negative_example_2"], inplace=True)

                flatten.append(sub_dataset)

    # Combine all DataFrames
    dataframe = pd.concat(flatten, axis=0)
    dataframe = dataframe.drop_duplicates(ignore_index=True)

    return dataframe


def build_dataset(dataframe):
    dataframe["prompt"] = dataframe.apply(build_prompt, axis=1)

    columns = ["prompt"]
    if "rule_violation" in dataframe:
        dataframe["completion"] = dataframe["rule_violation"].map(
            {
                1: POSITIVE_ANSWER,
                0: NEGATIVE_ANSWER,
            }
        )
        columns.append("completion")

    dataframe = dataframe[columns]
    dataset = Dataset.from_pandas(dataframe)
    dataset.to_pandas().to_csv("/kaggle/working/dataset_validation.csv", index=False)
    return dataset


def get_validation_dataframe_with_labels(data_path):
    """
    Get validation dataframe with true labels for evaluation
    """
    dataframe = get_dataframe_to_train_validation(data_path, mode='validation')
    return dataframe

## Evaluation Metrics and Visualization Functions

In [ ]:
%%writefile evaluation_metrics.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    roc_auc_score, confusion_matrix, classification_report, 
    roc_curve, precision_recall_curve, average_precision_score,
    accuracy_score, precision_score, recall_score, f1_score
)
import warnings
warnings.filterwarnings('ignore')

def comprehensive_evaluation_report(y_true, y_pred_proba, threshold=0.5, class_names=['No Violation', 'Violation'], model_name="Model"):
    """
    Generate comprehensive evaluation report with metrics and visualizations
    Exactly matches the reference unsloth-bnb implementation
    """
    
    # Convert probabilities to binary predictions
    y_pred_binary = (y_pred_proba >= threshold).astype(int)
    
    # Calculate basic metrics
    try:
        accuracy = accuracy_score(y_true, y_pred_binary)
        f1 = f1_score(y_true, y_pred_binary)
        precision = precision_score(y_true, y_pred_binary)
        recall = recall_score(y_true, y_pred_binary)
        auc = roc_auc_score(y_true, y_pred_proba)
        
        print("=" * 60)
        print(f"📊 SUMMARY METRICS ({model_name})")
        print("=" * 60)
        print(f"🎯 Accuracy:  {accuracy:.4f}")
        print(f"🎯 F1 Score:  {f1:.4f}")
        print(f"🎯 Precision: {precision:.4f}")
        print(f"🎯 Recall:    {recall:.4f}")
        print(f"🎯 ROC AUC:   {auc:.4f}")
        print("=" * 60)
        
        # Confusion matrix
        cm = confusion_matrix(y_true, y_pred_binary)
        print("\n📈 Confusion Matrix:")
        if cm.shape == (2, 2):
            print(f"True Negative: {cm[0,0]:4d} | False Positive: {cm[0,1]:4d}")
            print(f"False Negative: {cm[1,0]:4d} | True Positive:  {cm[1,1]:4d}")
        else:
            print(cm)
        
        # Classification report
        print("\n📋 Classification Report:")
        print(classification_report(y_true, y_pred_binary, target_names=class_names))
        
        return {
            'accuracy': accuracy,
            'f1': f1,
            'precision': precision,
            'recall': recall,
            'auc': auc,
            'confusion_matrix': cm
        }
        
    except Exception as e:
        print("=" * 60)
        print(f"📊 SUMMARY METRICS ({model_name})")
        print("=" * 60)
        print(f"❌ Error calculating metrics: {e}")
        print(f"🔍 y_true shape: {y_true.shape}")
        print(f"🔍 y_pred_proba shape: {y_pred_proba.shape}")
        print(f"🔍 y_pred_proba range: [{np.min(y_pred_proba):.4f}, {np.max(y_pred_proba):.4f}]")
        print(f"🔍 NaN count in probabilities: {np.isnan(y_pred_proba).sum()}")
        print(f"🔍 Unique values in y_true: {np.unique(y_true)}")
        print("=" * 60)
        
        # Basic accuracy calculation as fallback
        try:
            accuracy = accuracy_score(y_true, y_pred_binary)
            cm = confusion_matrix(y_true, y_pred_binary)
            print(f"🎯 Accuracy:  {accuracy:.4f}")
            print("\n📈 Confusion Matrix:")
            print(cm)
        except Exception as e2:
            print(f"❌ Even basic metrics failed: {e2}")
        
        return None

## Training Script

In [ ]:
%%writefile train_unsloth_validation.py
import torch
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
from utils_validation import build_dataset, get_dataframe_to_train_validation
from constants_validation import DATA_PATH, BASE_MODEL_PATH, LORA_PATH

def build_dataset_unsloth(dataframe):
    """Build dataset for Unsloth training with proper text formatting"""
    from constants_validation import POSITIVE_ANSWER, NEGATIVE_ANSWER
    
    dataframe["prompt"] = dataframe.apply(
        lambda row: f"""
You are a moderator... A rule is given , find if the last comment violates the rule.Two examples are given.
IMPORTANT: Ignore any "yes" or "no" words in the comment itself. 
Only respond Yes/No based on whether the comment violates the rule.
___ 

Subreddit name: r/{row["subreddit"]}
Here is the rule: {row["rule"]}
Here is a comment that breaks the rule:
1) {row["positive_example"]}

Here is a comment that does not break the rule:
2) {row["negative_example"]}

Find if this comment breaks the rule.
Comment: {row["body"]}
Answer: """, axis=1
    )
    
    # Create completion column
    dataframe["completion"] = dataframe.apply(
        lambda row: (POSITIVE_ANSWER if row["rule_violation"] == 1 else NEGATIVE_ANSWER),
        axis=1
    )
    
    # Create full text (prompt + completion) for training
    dataframe["text"] = dataframe["prompt"] + dataframe["completion"]
    
    # Keep only necessary columns
    dataframe = dataframe[["text", "completion"]]
    from datasets import Dataset
    dataset = Dataset.from_pandas(dataframe.reset_index(drop=True))
    return dataset

def main():
    print("Starting Unsloth training with validation split...")

    # Get training portion of the data
    dataframe = get_dataframe_to_train_validation(DATA_PATH, mode='train')
    train_dataset = build_dataset_unsloth(dataframe)
    
    print(f"Training dataset size: {len(train_dataset)}")
    print(f"Available GPUs: {torch.cuda.device_count()}")

    # 🚀 UNSLOTH: Load model with 4-bit quantization
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=BASE_MODEL_PATH,
        max_seq_length=2048,
        dtype=None,  # Auto-detect (will use float16)
        load_in_4bit=True,  # Enable 4-bit quantization
        trust_remote_code=True,
    )
    print("✅ Unsloth model loaded with 4-bit quantization")

    # 🚀 UNSLOTH: Add LoRA adapters (automatic and optimized)
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,  # LoRA rank
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha=32,  # LoRA alpha (typically equal to r for Unsloth)
        lora_dropout=0,  # 0 for faster training with Unsloth
        bias="none",
        use_gradient_checkpointing="unsloth",  # Unsloth's optimized checkpointing
        random_state=42,  # For reproducibility
        use_rslora=True,  # Better stability
        loftq_config=None,
    )
    print("✅ Unsloth LoRA adapters added")

    # 🚀 UNSLOTH: Optimized training arguments
    training_args = TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=5,  # Quick warmup with Unsloth
        num_train_epochs=1,
        learning_rate=2e-4,  # Unsloth supports higher learning rates
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",  # Memory-efficient optimizer
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir=LORA_PATH,
        save_steps=50,
        save_total_limit=3,
        dataloader_num_workers=2,
        remove_unused_columns=False,
        push_to_hub=False,
        report_to="none",
    )
    print("✅ Unsloth training arguments configured")

    # 🚀 UNSLOTH: Fast SFT Trainer (optimized for Unsloth)
    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        dataset_text_field="text",  # The text column containing the full conversation
        max_seq_length=2048,
        dataset_num_proc=2,
        packing=False,
        args=training_args,
    )
    print("✅ Unsloth SFT Trainer created")

    # 🚀 Training with Unsloth (2-20x faster than standard)
    print("🚀 Starting Unsloth training...")
    trainer.train()
    print("✅ Unsloth training completed!")

    # 🚀 UNSLOTH: Save merged model for faster inference
    merged_folder = "/kaggle/working/Merged_unsloth_model"
    model.save_pretrained_merged(merged_folder, tokenizer, save_method="merged_16bit")

    # 🚀 UNSLOTH: Save LoRA adapters
    model.save_pretrained(LORA_PATH)
    tokenizer.save_pretrained(LORA_PATH)
    
    print(f"✅ LoRA adapters saved to {LORA_PATH}")
    print(f"✅ Merged model saved to {merged_folder}")
    print("🎯 Unsloth training complete! Ready for validation.")

if __name__ == "__main__":
    main()

## Validation Inference Script

In [ ]:
%%writefile inference_validation.py
import os
os.environ["VLLM_USE_V1"] = "0"

import vllm
import torch
import pandas as pd
import numpy as np
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
from vllm.lora.request import LoRARequest
from utils_validation import build_dataset, get_validation_dataframe_with_labels
from constants_validation import BASE_MODEL_PATH, LORA_PATH, DATA_PATH, POSITIVE_ANSWER, NEGATIVE_ANSWER
from evaluation_metrics import comprehensive_evaluation_report


def run_validation_inference():
    """Run inference on validation set and return predictions with true labels"""
    
    print("Loading validation dataset...")
    validation_df = get_validation_dataframe_with_labels(DATA_PATH)
    
    # Build dataset for inference (only prompts needed)
    validation_dataset = build_dataset(validation_df)
    texts = validation_dataset["prompt"]
    
    # Get true labels
    y_true = validation_df["rule_violation"].values
    
    print(f"Validation set size: {len(texts)}")
    print(f"Class distribution: {np.bincount(y_true)}")
    
    print("Initializing vLLM model...")
    llm = vllm.LLM(
        BASE_MODEL_PATH,
        quantization="gptq",
        tensor_parallel_size=1,
        gpu_memory_utilization=0.98,
        trust_remote_code=True,
        dtype="half",
        enforce_eager=True,
        max_model_len=2836,
        disable_log_stats=True,
        enable_prefix_caching=True,
        enable_lora=True,
        max_lora_rank=64,
    )
    
    tokenizer = llm.get_tokenizer()
    mclp = MultipleChoiceLogitsProcessor(tokenizer, choices=[POSITIVE_ANSWER, NEGATIVE_ANSWER])
    
    print("Running inference...")
    outputs = llm.generate(
        texts,
        vllm.SamplingParams(
            skip_special_tokens=True,
            max_tokens=1,
            logits_processors=[mclp],
            logprobs=2,
        ),
        use_tqdm=True,
        lora_request=LoRARequest("default", 1, LORA_PATH)
    )
    
    # Extract predictions and probabilities
    log_probs = [
        {lp.decoded_token: lp.logprob for lp in out.outputs[0].logprobs[0].values()}
        for out in outputs
    ]
    
    predictions_df = pd.DataFrame(log_probs)
    
    # Convert log probabilities to probabilities
    yes_logprobs = predictions_df[POSITIVE_ANSWER].values
    no_logprobs = predictions_df[NEGATIVE_ANSWER].values
    
    # Calculate softmax to get proper probabilities
    yes_probs = np.exp(yes_logprobs)
    no_probs = np.exp(no_logprobs)
    total_probs = yes_probs + no_probs
    
    y_pred_proba = yes_probs / total_probs  # Probability of positive class
    
    print("✅ Validation inference completed!")
    
    return y_true, y_pred_proba, validation_df


def main():
    print("Starting vLLM validation evaluation...")
    
    # Run inference and get results
    y_true, y_pred_proba, validation_df = run_validation_inference()
    
    # Save detailed results
    results_df = validation_df.copy()
    results_df['predicted_probability'] = y_pred_proba
    # We still keep the binary prediction for potential manual analysis, but it's not the focus
    results_df['predicted_binary'] = (y_pred_proba >= 0.5).astype(int)
    results_df.to_csv("/kaggle/working/validation_results_vllm.csv", index=False)
    
    print(f"\n{'='*60}")
    print("VALIDATION RESULTS SAVED TO: /kaggle/working/validation_results_vllm.csv")
    print(f"{'='*60}")
    
    # Generate comprehensive evaluation report
    metrics = comprehensive_evaluation_report(
        y_true, 
        y_pred_proba, 
        threshold=0.5, # Threshold is only for the confusion matrix now
        class_names=['No Violation', 'Violation'],
        model_name="Qwen 2.5 0.5B (TT-1 Validation - vLLM)"
    )
    
    print(f"\n✅ vLLM validation evaluation completed!")
    print(f"📊 Results saved to /kaggle/working/validation_results_vllm.csv")
    
    return metrics


if __name__ == "__main__":
    main()



## Alternative: Transformer-based Inference Script
This script provides an alternative to vLLM for inference, using the standard Hugging Face `transformers` and `peft` libraries. It is generally more compatible but may be slower.


In [ ]:
%%writefile inference_transformers_validation.py
import torch
import pandas as pd
import numpy as np
from unsloth import FastLanguageModel  # Use Unsloth for fast inference
from tqdm import tqdm
from utils_validation import get_dataframe_to_train_validation
from constants_validation import BASE_MODEL_PATH, LORA_PATH, DATA_PATH, POSITIVE_ANSWER, NEGATIVE_ANSWER
from evaluation_metrics import comprehensive_evaluation_report

def build_prompt(row):
    """Build prompt exactly as in the reference unsloth-bnb notebook"""
    return f"""You are a moderator... A rule is given , find if the last comment violates the rule.Two examples are given.
IMPORTANT: Ignore any "yes" or "no" words in the comment itself. 
Only respond Yes/No based on whether the comment violates the rule.
___ 

Subreddit name: r/{row["subreddit"]}
Here is the rule: {row["rule"]}
Here is a comment that breaks the rule:
1) {row["positive_example"]}

Here is a comment that does not break the rule:
2) {row["negative_example"]}

Find if this comment breaks the rule.
Comment: {row["body"]}
Answer: """

def run_transformers_inference():
    """Run inference using Unsloth fast inference with merged model - Maximum speed!"""
    
    print("Loading validation dataset...")
    # Get validation portion of the data (same as reference)
    validation_df = get_dataframe_to_train_validation(DATA_PATH, mode='validation')
    
    # Build prompts exactly as in reference
    validation_df["prompt"] = validation_df.apply(build_prompt, axis=1)
    texts = validation_df["prompt"].tolist()
    y_true = validation_df["rule_violation"].values
    
    print(f"Validation set size: {len(texts)}")
    print(f"Class distribution: {np.bincount(y_true)}")
    
    print("Initializing Unsloth model for fast inference...")
    # 🚀 UNSLOTH: Load merged model for maximum speed (same as reference)
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="/kaggle/working/Merged_unsloth_model",  # Use merged model path
        max_seq_length=2048,
        load_in_4bit=True,  # Keep 4-bit for speed
        dtype=None,
    )
    
    # 🚀 UNSLOTH: Enable fast inference mode
    FastLanguageModel.for_inference(model)
    
    # Get token IDs for "Yes" and "No" - exactly as in reference
    yes_token_id = tokenizer.convert_tokens_to_ids("Yes")
    no_token_id = tokenizer.convert_tokens_to_ids("No")
    
    print(f"🎯 Token IDs: Yes={yes_token_id}, No={no_token_id}")
    
    # Validate token IDs
    if yes_token_id is None or no_token_id is None:
        print("❌ Token ID detection failed!")
        # Try alternative approaches
        print("Trying alternative token detection...")
        yes_token_id = tokenizer.encode("Yes")[0] if tokenizer.encode("Yes") else None
        no_token_id = tokenizer.encode("No")[0] if tokenizer.encode("No") else None
        print(f"🔄 Alternative Token IDs: Yes={yes_token_id}, No={no_token_id}")
        
        if yes_token_id is None or no_token_id is None:
            print("❌ Critical error: Cannot find Yes/No token IDs")
            return None, None, None
    
    predictions = []
    probabilities = []
    batch_size = 8  # Same as reference
    
    print("🚀 Running fast inference with Unsloth...")
    
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        
        # 🚀 UNSLOTH: Optimized tokenization and inference (exact same as reference)
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            # 🚀 UNSLOTH: Fast forward pass
            outputs = model(**inputs)
            next_token_logits = outputs.logits[:, -1, :]  # Get last token logits
            
            # Get probabilities for "Yes" and "No" tokens (exact same as reference)
            yes_logits = next_token_logits[:, yes_token_id]
            no_logits = next_token_logits[:, no_token_id]
            
            # Convert to probabilities using softmax over Yes/No only (exact same as reference)
            combined_logits = torch.stack([no_logits, yes_logits], dim=1)  # [batch, 2]
            probs = torch.softmax(combined_logits, dim=1)  # [batch, 2]
            
            # Extract predictions and probabilities (exact same as reference)
            batch_predictions = torch.argmax(probs, dim=1).cpu().numpy()
            batch_probabilities = probs[:, 1].cpu().numpy()  # Probability of "Yes" (violation)
            
            predictions.extend(batch_predictions.tolist())
            probabilities.extend(batch_probabilities.tolist())
    
    y_pred_proba = np.array(probabilities)
    
    print("✅ Transformers validation inference completed!")
    print(f"🔍 Sample probabilities: {y_pred_proba[:5]}")
    print(f"🔍 Probability range: [{y_pred_proba.min():.4f}, {y_pred_proba.max():.4f}]")
    print(f"🔍 NaN count: {np.isnan(y_pred_proba).sum()}")
    
    return y_true, y_pred_proba, validation_df

def main():
    print("Starting Transformers validation evaluation...")
    
    # Run inference
    result = run_transformers_inference()
    if result[0] is None:
        print("❌ Inference failed due to token ID issues")
        return
        
    y_true, y_pred_proba, validation_df = result
    
    # Check for NaN values
    if np.isnan(y_pred_proba).any():
        print(f"❌ Found {np.isnan(y_pred_proba).sum()} NaN values in probabilities")
        print("Replacing NaN values with 0.5 (neutral probability)")
        y_pred_proba = np.nan_to_num(y_pred_proba, nan=0.5)
    
    # Save detailed results
    results_df = validation_df.copy()
    results_df['predicted_probability'] = y_pred_proba
    results_df['predicted_binary'] = (y_pred_proba >= 0.5).astype(int)
    results_df.to_csv("/kaggle/working/validation_results_transformers.csv", index=False)
    
    print(f"\n{'='*60}")
    print("VALIDATION RESULTS SAVED TO: /kaggle/working/validation_results_transformers.csv")
    print(f"{'='*60}")
    
    # Generate comprehensive evaluation report
    comprehensive_evaluation_report(
        y_true, 
        y_pred_proba, 
        threshold=0.5,
        class_names=['No Violation', 'Violation'],
        model_name="Qwen 2.5 0.5B (Unsloth Transformers Validation)"
    )
    
    print(f"\n✅ Transformers validation evaluation completed!")

if __name__ == "__main__":
    main()

## Accelerate Configuration

In [ ]:
%%writefile accelerate_config_validation.yaml
compute_environment: LOCAL_MACHINE
debug: false
deepspeed_config:
  gradient_accumulation_steps: 4
  gradient_clipping: 1.0
  train_batch_size: 64
  train_micro_batch_size_per_gpu: 4
  
  zero_stage: 2
  offload_optimizer_device: none
  offload_param_device: none
  zero3_init_flag: false
  
  stage3_gather_16bit_weights_on_model_save: false
  stage3_max_live_parameters: 1e8
  stage3_max_reuse_distance: 1e8
  stage3_prefetch_bucket_size: 5e7
  stage3_param_persistence_threshold: 1e5
  
  zero_allow_untested_optimizer: true
  zero_force_ds_cpu_optimizer: false
  
  fp16:
    enabled: true
    loss_scale: 0
    initial_scale_power: 16
    loss_scale_window: 1000
    hysteresis: 2
    min_loss_scale: 1
  
distributed_type: DEEPSPEED
downcast_bf16: 'no'
dynamo_config:
  dynamo_backend: INDUCTOR
  dynamo_use_fullgraph: false
  dynamo_use_dynamic: false
enable_cpu_affinity: false
machine_rank: 0
main_training_function: main
mixed_precision: fp16
num_machines: 1
num_processes: 2
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false

## Execution: Training and Validation

In [ ]:
# Step 1: Run Unsloth training on the controlled portion of training data
!python train_unsloth_validation.py

In [ ]:
# Step 2: Run Validation Inference
# Choose ONE of the following methods to run validation.

# Method 1: vLLM (Fast, recommended if hardware is compatible)
!python inference_validation.py

# Method 2: Standard Transformers (Slower, more compatible)
# !python inference_transformers_validation.py

## Results Analysis and Visualization

In [ ]:
# Load and examine validation results
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- CHOOSE YOUR RESULTS SOURCE ---
# Change this to 'transformers' to analyze the results from the transformers script
# validation_source = 'vllm' 
validation_source = 'transformers'

# --- DO NOT EDIT BELOW THIS LINE ---

results_file = f'/kaggle/working/validation_results_{validation_source}.csv'
threshold_file = f'/kaggle/working/threshold_analysis_{validation_source}.csv'

if not os.path.exists(results_file):
    raise FileNotFoundError(f"Results file not found: {results_file}. Please run the corresponding validation script first.")

# Load validation results
validation_results = pd.read_csv(results_file)
if os.path.exists(threshold_file):
    threshold_analysis = pd.read_csv(threshold_file)
else:
    threshold_analysis = None # May not exist for transformers script

print(f"--- Analyzing results from: {validation_source.upper()} ---")
print("\nValidation Results Overview:")
print(f"Total validation samples: {len(validation_results)}")
print(f"Actual class distribution:")
print(validation_results['rule_violation'].value_counts().sort_index())
print(f"\nPredicted class distribution (threshold=0.5):")
print(validation_results['predicted_binary'].value_counts().sort_index())

# Display sample of results
print(f"\nSample of validation results:")
print(validation_results[['body', 'rule_violation', 'predicted_probability', 'predicted_binary']].head())


In [ ]:
# Additional Analysis: Prediction Distribution
plt.figure(figsize=(15, 5))

# Subplot 1: Prediction probability distribution
plt.subplot(1, 3, 1)
plt.hist(validation_results['predicted_probability'], bins=50, alpha=0.7, edgecolor='black')
plt.title('Distribution of Predicted Probabilities', fontweight='bold')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

# Subplot 2: Prediction probabilities by true class
plt.subplot(1, 3, 2)
no_violation = validation_results[validation_results['rule_violation'] == 0]['predicted_probability']
violation = validation_results[validation_results['rule_violation'] == 1]['predicted_probability']

plt.hist(no_violation, bins=30, alpha=0.7, label='No Violation (True)', color='blue', edgecolor='black')
plt.hist(violation, bins=30, alpha=0.7, label='Violation (True)', color='red', edgecolor='black')
plt.title('Predicted Probabilities by True Class', fontweight='bold')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 3: Box plot of predictions by true class
plt.subplot(1, 3, 3)
data_for_boxplot = [no_violation, violation]
labels = ['No Violation', 'Violation']
plt.boxplot(data_for_boxplot, labels=labels)
plt.title('Prediction Distribution by True Class', fontweight='bold')
plt.ylabel('Predicted Probability')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate and display calibration metrics
from sklearn.calibration import calibration_curve

# Reliability diagram (calibration curve)
fraction_of_positives, mean_predicted_value = calibration_curve(
    validation_results['rule_violation'], 
    validation_results['predicted_probability'], 
    n_bins=10
)

plt.figure(figsize=(8, 6))
plt.plot(mean_predicted_value, fraction_of_positives, "s-", label="Model")
plt.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives')
plt.title('Calibration Plot (Reliability Diagram)', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Print summary statistics
print("\\nSummary Statistics:")
print(f"Mean predicted probability: {validation_results['predicted_probability'].mean():.4f}")
print(f"Std predicted probability: {validation_results['predicted_probability'].std():.4f}")
print(f"Min predicted probability: {validation_results['predicted_probability'].min():.4f}")
print(f"Max predicted probability: {validation_results['predicted_probability'].max():.4f}")

# Most confident correct and incorrect predictions
correct_predictions = validation_results[validation_results['rule_violation'] == validation_results['predicted_binary']]
incorrect_predictions = validation_results[validation_results['rule_violation'] != validation_results['predicted_binary']]

print(f"\\nModel Performance Summary:")
print(f"Correct predictions: {len(correct_predictions)} ({len(correct_predictions)/len(validation_results)*100:.1f}%)")
print(f"Incorrect predictions: {len(incorrect_predictions)} ({len(incorrect_predictions)/len(validation_results)*100:.1f}%)")

if len(incorrect_predictions) > 0:
    print(f"\\nMost confident incorrect predictions:")
    # For incorrect predictions, show those with highest confidence (furthest from 0.5)
    incorrect_predictions['confidence'] = np.abs(incorrect_predictions['predicted_probability'] - 0.5)
    most_confident_wrong = incorrect_predictions.nlargest(3, 'confidence')
    for idx, row in most_confident_wrong.iterrows():
        print(f"True: {row['rule_violation']}, Pred: {row['predicted_binary']}, Prob: {row['predicted_probability']:.3f}")
        print(f"Comment: {row['body'][:100]}...")
        print(f"Rule: {row['rule'][:100]}...")
        print("-" * 50)

## Configuration Summary

This validation notebook provides a comprehensive evaluation of the TT-1 Qwen 2.5 0.5B model, accelerated with **Unsloth**.

### Key Features:
1. **Unsloth-Powered Training**: Leverages Unsloth for significantly faster fine-tuning and reduced memory usage.
2. **Dual Inference Options**:
   - **vLLM**: High-performance inference for quick validation.
   - **Transformers**: Standard, compatible inference as a reliable alternative.
3. **Controlled Training Split**: Uses 70% of training data for training, 30% for validation, ensuring no data leakage.
4. **Comprehensive Metrics**: AUC, confusion matrix, classification report, ROC curve, and precision-recall curve.
5. **Flexible Analysis**: The analysis section can dynamically load and visualize results from either the vLLM or Transformers output.

### Configuration:
- **Training Framework**: **Unsloth**
- **Training Percentage**: 70% of original training data
- **Validation Percentage**: 30% of original training data  
- **Model**: Qwen 2.5 0.5B with 4-bit quantization
- **LoRA Configuration**: r=16, alpha=32
- **Training**: 1 epoch with adamw_8bit optimizer

### Output Files:
- `validation_results_vllm.csv` / `validation_results_transformers.csv`: Detailed per-sample results.
- `threshold_analysis_vllm.csv`: Performance metrics across different thresholds (vLLM only).
- Comprehensive visualizations and analysis charts.

This setup allows for robust, accelerated validation and provides deep insights for model improvement.


# TT-1 Validation Notebook (Unsloth Edition): Qwen 2.5 0.5B Model

This notebook trains a Qwen 2.5 0.5B model using **Unsloth** for accelerated performance. It trains on a controlled percentage of the training data and validates on the remaining portion.

It provides two options for inference:
1.  **vLLM**: For high-throughput, fast inference (recommended).
2.  **Transformers**: A standard, more compatible inference method.

**Evaluation Metrics:**
- AUC Score
- Confusion Matrix 
- Classification Report
- ROC Curve & Precision-Recall Curve

**Workflow:**
- **Train**: Fine-tune the model with Unsloth on 70% of the training data.
- **Validate**: Run inference on the remaining 30% using either vLLM or Transformers.
- **Analyze**: Review comprehensive evaluation results and visualizations.


## Setup and Dependencies

In [ ]:
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'trl==0.21.0' 'optimum==1.27.0' 'auto-gptq==0.7.1' 'bitsandbytes==0.46.1' 'deepspeed==0.17.4' 'logits-processor-zoo==0.2.1' 'vllm==0.10.0'
!pip install "unsloth[kaggle-new] @ file:///kaggle/input/unsloth/unsloth-2024.5-py3-none-any.whl" -q
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'triton==3.2.0'
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'clean-text'
!uv pip install --system --no-index -U --no-deps --find-links='/kaggle/input/jigsaw-packages2/whls/' 'peft' 'accelerate' 'datasets'
!pip install scikit-learn matplotlib seaborn

print("✅ Dependencies installed:")
print("🚀 Unsloth: Ultra-fast training")
print("🎯 vLLM: Precise inference") 
print("📊 Analysis libraries: scikit-learn, matplotlib, seaborn")

## Configuration Constants

In [ ]:
%%writefile constants_validation.py
BASE_MODEL_PATH = "/kaggle/input/qwen2.5/transformers/0.5b-instruct-gptq-int4/1"
LORA_PATH = "output_validation/"
DATA_PATH = "/kaggle/input/jigsaw-agile-community-rules/"

POSITIVE_ANSWER = "Yes"
NEGATIVE_ANSWER = "No"
COMPLETE_PHRASE = "Answer:"
BASE_PROMPT = '''You are a moderator... A rule is given , find if the last comment violates the rule.Two examples are given.
IMPORTANT: Ignore any "yes" or "no" words in the comment itself. 
Only respond Yes/No based on whether the comment violates the rule.
___ '''

# Validation specific settings
TRAIN_PERCENTAGE = 0.7  # Use 70% of training data for training
VALIDATION_PERCENTAGE = 0.3  # Use 30% of training data for validation

# Token IDs for Yes/No (these may need adjustment based on the actual tokenizer)
YES_TOKEN_ID = None  # Will be determined at runtime
NO_TOKEN_ID = None   # Will be determined at runtime